In [2]:
#import requests
import pandas as pd
import numpy as np
import re
import sys
import pickle
import time

from datetime import datetime
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By

option = webdriver.ChromeOptions()
## stop loaing the images to enhance perfomance
chrome_prefs = {}
#conda install -c conda-forge selenium
option.experimental_options["prefs"] = chrome_prefs
chrome_prefs["profile.default_content_settings"] = {"images": 2}
chrome_prefs["profile.managed_default_content_settings"] = {"images": 2}

driver = webdriver.Chrome(options=option)


In [2]:
# Function to read each url
def read_url_imdb(url):
    driver.get(url)
    #response = requests.get(url, timeout=10)
    #page = response.text
    page = driver.page_source
    #soup = BeautifulSoup(page, "lxml")
    soup = BeautifulSoup(page, 'html.parser')
    return soup

In [4]:
# Using 100 pages of 50 movies per page, 5000 movies, sorted by IMDB popularity
url_list_imdb=[]
for i in range(1,10000,50):
    url_list_imdb.append("https://www.imdb.com/search/title/?genres=comedy&start="+str(i)+"&explore=title_type,genres&ref_=adv_nxt")

In [5]:
# Confirming length of list is 100 as 100 pages
len(url_list_imdb)

200

In [7]:
# Creating a list of the titles of the top IMDB movies
movie_titles_imdb = []
imdb_ratings=[]
MPAA_ratings=[]
genres_imdb=[]
runtimes_imdb = []
movie_urls_imdb = []
movie_actors_imdb = []
count = 1
for url in url_list_imdb:

    while True:
        try:
            data_movies_imdb = read_url_imdb(url)
        except:
            continue
        break
        

    for entry in data_movies_imdb.find_all("div", {"class": "lister-item-content"}):
        movie_title = entry.find('a').contents[0]
        movie_titles_imdb.append(movie_title)

# Creating a list of ratings of the top IMDB movies
    for entry in data_movies_imdb.find_all("div",{"class": "lister-item-content"}):
        try:
            if entry.find('div', {"class":"ratings-bar"}):
                rating = float(entry.find('div',{'class':'ratings-bar'}).find('strong').get_text())
                imdb_ratings.append(rating)
            else:
                imdb_ratings.append(np.nan)
        except:
            imdb_ratings.append(np.nan)

# Collecting MPAA ratings for movies from IMDB 
    for entry in data_movies_imdb.find_all("div", {"class":"lister-item-content"}):
        try:
            if entry.find("p", {"class":"text-muted"}):
                mpaa = entry.find("span", {"class":"certificate"}).get_text()
                MPAA_ratings.append(mpaa)
            else:
                MPAA_ratings.append(np.nan)
        except:
            MPAA_ratings.append(np.nan)

# Collecting genres for movies from IMDB
    genre=data_movies_imdb.find_all('span',{"class":"genre"})
    for item in [g.get_text() for g in genre]:
        genres_imdb.append(str(item).replace('\n','').strip())
        
# Collecting runtimes for movies from IMDB
    for entry in data_movies_imdb.find_all("p", {"class": "text-muted"}):
        if entry.find('span')==None:
            pass
        elif entry.find_all('span')[0]['class']==['certificate'] and entry.find_all('span')[2]['class']!=['genre']:
            runtime_digits = re.findall(r"\d+", entry.find_all('span')[2].get_text()).pop()
            runtimes_imdb.append(int(runtime_digits))
        elif entry.find_all('span')[0]['class']==['runtime']:
            runtime_digits = re.findall(r"\d+", entry.find_all('span')[0].get_text()).pop()
            runtimes_imdb.append(int(runtime_digits))
        else:
            runtimes_imdb.append(np.nan)

# Collecting movie urls
    for entry in data_movies_imdb.find_all("div",{"class": "lister-item-content"}):
        try: 
            if entry.find('h3', {'class':'lister-item-header'}).find('span',{'class': 'lister-item-index unbold text-primary'}):
                movie_url_imdb = entry.find('h3', {'class':'lister-item-header'}).find('a')['href']
                movie_urls_imdb.append(movie_url_imdb)
            else:
                movie_urls_imdb.append(np.nan)
        except:
            movie_urls_imdb.append(np.nan)


# Scraping actors for each movie

    for table in data_movies_imdb.find_all('div', {"class":"lister-item mode-advanced"}):
        actors = []
        if table.find(text=re.compile("Stars:")):
            stars = table.find(text=re.compile("Stars:"))
            actors.append(stars.findNext().get_text().replace(',','').replace('.','').replace("'",'').replace('-',''))
            actors.append(stars.findNext().findNext().get_text().replace(',','').replace('.','').replace("'",'').replace('-',''))            
            actors.append(stars.findNext().findNext().findNext().get_text().replace(',','').replace('.','').replace("'",'').replace('-',''))         
            actors.append(stars.findNext().findNext().findNext().findNext().get_text().replace(',','').replace('.','').replace("'",'').replace('-',''))
        elif table.find(text=re.compile("Star:")):
            star = table.find(text=re.compile("Star:"))
            actors = star.findNext().get_text().replace(',','').replace('.','').replace("'",'').replace('-','')
        else:
            actors = 'TBD'

        movie_actors_imdb.append(actors)

In [8]:
# Confirm # of titles = 50*100
print(len(movie_titles_imdb))
print(len(imdb_ratings))
print(len(MPAA_ratings))
print(len(movie_actors_imdb))
print(len(runtimes_imdb))
#print(runtimes_imdb)
print(len(movie_urls_imdb))

10000
10000
10000
10000
10000
10000


In [10]:


# Find no. of unique genres and combine 'Music' and 'Musical' categories
from collections import defaultdict
unique_genres_imdb = []
for entry in genres_imdb:
        entry_list = str(entry).replace(' ','').replace('Musical','Music').replace('-','').split(',')
        for genre in entry_list:
            if genre not in unique_genres_imdb:
                unique_genres_imdb.append(genre)
unique_genres_imdb = sorted(unique_genres_imdb)
print(unique_genres_imdb)

# Converting genres to dummy variable
genre_set=['Action','Adventure','Animation','Biography','Comedy','Crime','Drama','Family','Fantasy','History','Horror',
      'Music','Mystery','Romance','SciFi','Sport','Thriller','War','Western']
Action,Adventure,Animation,Biography,Comedy,Crime,Drama,Family,Fantasy,History,Horror,Music,Mystery,Romance,SciFi,Sport,Thriller,War,Western=([] for i in range(len(genre_set)))

# Creates empty list for each genre represented in the data
genre_list=[Action,Adventure,Animation,Biography,Comedy,Crime,Drama,Family,Fantasy,History,Horror,Music,Mystery,Romance,SciFi,Sport,Thriller,War,Western]
for g in genres_imdb:
    for c in genre_set:
        if c in g:
            genre_list[genre_set.index(c)].append(1)
        else:
            genre_list[genre_set.index(c)].append(0)


# Collecting runtimes for movies from IMDB


print(runtimes_imdb)

['Action', 'Adventure', 'Animation', 'Biography', 'Comedy', 'Crime', 'Documentary', 'Drama', 'Family', 'Fantasy', 'FilmNoir', 'GameShow', 'History', 'Horror', 'Music', 'Mystery', 'News', 'RealityTV', 'Romance', 'SciFi', 'Short', 'Sport', 'TalkShow', 'Thriller', 'War', 'Western']
[30, 115, 45, nan, 22, nan, 52, 96, 30, 22, 108, 46, 30, 43, 134, 22, nan, 132, 93, 60, nan, 22, 43, nan, 130, nan, nan, 22, 22, 23, 30, 30, 24, 353, 22, 132, 22, 72, 43, 22, 22, 99, 30, 44, 161, 44, 40, 22, nan, 28, 86, 22, 22, 180, 22, 24, 25, 102, 127, 57, 43, 22, 92, 30, nan, 45, 100, 113, 88, 114, 44, 22, 22, 43, 106, 112, 110, 82, 22, 22, 48, 90, 102, 94, 148, 30, 22, nan, 145, 113, 105, 350, 44, nan, 108, 110, 90, 60, 26, 22, 60, nan, 103, 70, 24, 101, 60, 84, nan, 45, 112, 97, 105, 22, 25, 100, 104, 30, 30, 22, nan, 22, 57, 109, 59, 101, 22, 95, 22, 96, 102, 45, 23, 106, 97, 100, 116, 113, 30, nan, 22, 123, 103, 44, 121, 97, 108, 30, 119, 44, 100, 95, nan, 117, 32, nan, 30, 104, 108, 30, 99, 25, 30, 136

In [11]:
movies_imdb_subset1 = {'Title':movie_titles_imdb,'IMDB Rating':imdb_ratings, 'MPAA Rating':MPAA_ratings, 'Runtime':runtimes_imdb, 'Action':Action,'Adventure':Adventure,'Animation':Animation,'Biography':Biography,'Comedy':Comedy,'Crime':Crime,'Drama':Drama,'Family':Family,'Fantasy':Fantasy,'History':History,'Horror':Horror,'Music':Music,'Mystery':Mystery,'Romance':Romance,'SciFi':SciFi,'Sport':Sport,'Thriller':Thriller,'War':War,'Western':Western}
movies_imdb_subset1 = pd.DataFrame(movies_imdb_subset1)

In [12]:
movies_imdb_subset1.to_csv('movies_imdb_subset1.csv')

In [14]:
individual_movie_urls_imdb = ['https://www.imdb.com' + i for i in movie_urls_imdb]



In [15]:
import time

movie_budgets= []
i = 1
# counts to check
live_counts = 0
count = 0
page_visits = 0
for url in individual_movie_urls_imdb:
    while True:
        try:
            data_movies_imdb = read_url_imdb(url)
            print("count " + str(live_counts) + " " + url, end=' ')
            live_counts += 1
        except:
            print("I had an issue connecting, sleep for 15 seconds.")
            time.sleep(15)
            continue
        break
    try:
        if data_movies_imdb.find_all('span', {'class':'ipc-metadata-list-item__list-content-item'})[3]:
            budget_table = data_movies_imdb.find_all('span', {'class':'ipc-metadata-list-item__list-content-item'})[3]
            budget_table = budget_table.get_text().replace('$','').replace(',','')
            budget = re.sub('\D','',budget_table)
            i += 1
            movie_budgets.append(budget)
            if i % 100 == 0:
                print(budget)
                with open('movie_budgets.pkl', 'wb') as f:
                    pickle.dump(movie_budgets, f)
        else:
            movie_budgets.append(np.nan)
    except:
        movie_budgets.append(np.nan)




count 0 https://www.imdb.com/title/tt10986410/?ref_=adv_li_tt count 1 https://www.imdb.com/title/tt6264654/?ref_=adv_li_tt count 2 https://www.imdb.com/title/tt7767422/?ref_=adv_li_tt count 3 https://www.imdb.com/title/tt12851524/?ref_=adv_li_tt count 4 https://www.imdb.com/title/tt0098904/?ref_=adv_li_tt count 5 https://www.imdb.com/title/tt11012066/?ref_=adv_li_tt count 6 https://www.imdb.com/title/tt14602326/?ref_=adv_li_tt count 7 https://www.imdb.com/title/tt0107120/?ref_=adv_li_tt count 8 https://www.imdb.com/title/tt7908628/?ref_=adv_li_tt count 9 https://www.imdb.com/title/tt0386676/?ref_=adv_li_tt count 10 https://www.imdb.com/title/tt8847712/?ref_=adv_li_tt count 11 https://www.imdb.com/title/tt1586680/?ref_=adv_li_tt count 12 https://www.imdb.com/title/tt14369056/?ref_=adv_li_tt count 13 https://www.imdb.com/title/tt1219024/?ref_=adv_li_tt count 14 https://www.imdb.com/title/tt3228774/?ref_=adv_li_tt count 15 https://www.imdb.com/title/tt0108778/?ref_=adv_li_tt count 16 http

In [16]:
#checks
#movie_budgets_init = movie_budgets
#print(len(movie_budgets))
#print(movie_budgets_init)
print(len(movie_budgets))
print(movie_budgets)
print(movie_budgets.count(''))


10000
['', '120000000', '', '1851', '22', '1851', '52', '', '30', '', '25000000', '', '30', '', '', '', nan, '185000000', '42273382', '1', nan, '99', '', nan, '', nan, '2391', '', '', '', '30', '', '2001', '169', '', '', '', '169', '43', '', '', '', '', '', '9', '', '', '', nan, '', '', '', '', '', '', '', '', '142', '116763166', nan, '', '', '', '169', nan, '', '', '153', '', '', '44', '', '', '', '2351', '', '7000000', '', '', '22', '', '25', '', '2', nan, '', '', '169', '2391', '153', '', '', '', nan, '229599', '', '1362067', '', nan, '', '', nan, '', '1080', '24', '', '', '124', nan, '', '', '2001', '36000000', '22', '', '', '', '30', nan, '', '169', '', '57', '', '', '', '4', '49010641', '22', '25000000', '', '45', '23', '', '', '70000000', '', '', '', '2351', '', '', '', '', '', '', '', '30', '', '', '', '', '1851', '12163366', '32', nan, '', '', '', '', '25000000', '', nan, '85000000', '', '', '3', '', nan, '', '169', '', '1781', '150000000', '128', '', '3', '', '2', '', '', '',

In [18]:
#print(len(movie_actors_imdb))

In [19]:
# Collecting data from the-numbers.com
url_list_thenumbers=[]
for num in range(5):
    url_list_thenumbers.append('https://www.the-numbers.com/box-office-star-records/domestic/lifetime-acting/top-grossing-leading-stars/{}01'.format(num))

highest_grossing_actors = []

#for url in url_list_thenumbers:     
for url in url_list_thenumbers:
    entries=read_url_imdb(url).find_all('td', {'class':'data'})
    for entry in entries:
        actor = entry.findNext().get_text().replace(',','').replace('.','').replace("'",'').replace('-','')
        highest_grossing_actors.append(actor)

In [20]:
print(len(highest_grossing_actors)) # should be 500
print(highest_grossing_actors)

500
['Samuel L Jackson', 'Robert Downey Jr', 'Scarlett Johansson', 'Tom Hanks', 'Bradley Cooper', 'Harrison Ford', 'Chris Evans', 'Tom Cruise', 'Chris Hemsworth', 'Zoe Saldana', 'Chris Pratt', 'Johnny Depp', 'Eddie Murphy', 'Will Smith', 'Vin Diesel', 'Dwayne Johnson', 'Mark Ruffalo', 'Don Cheadle', 'Matt Damon', 'Jeremy Renner', 'Emma Watson', 'Bruce Willis', 'Adam Sandler', 'Chadwick Boseman', 'Ben Stiller', 'Nicolas Cage', 'Karen Gillan', 'Steve Carell', 'Jim Carrey', 'Josh Brolin', 'Leonardo DiCaprio', 'Jennifer Lawrence', 'Robin Williams', 'Benedict Cumberbatch', 'Liam Neeson', 'Robert De Niro', 'Tom Holland', 'Daniel Radcliffe', 'Mark Wahlberg', 'Hugh Jackman', 'Sandra Bullock', 'Brad Pitt', 'Gwyneth Paltrow', 'Cameron Diaz', 'Denzel Washington', 'Ian McKellen', 'Elizabeth Olsen', 'Ewan McGregor', 'Sebastian Stan', 'Julia Roberts', 'Dave Bautista', 'Tom Hiddleston', 'Rupert Grint', 'Ben Affleck', 'Paul Bettany', 'Mel Gibson', 'Kevin Hart', 'John Travolta', 'Owen Wilson', 'Mark Ha

In [21]:
# edit this code
actor_scores_per_movie = []
for actor_list in movie_actors_imdb:
    actor_score = 0
    for actor in actor_list:
        if actor in highest_grossing_actors:
            actor_score += 1
    actor_scores_per_movie.append(actor_score)
print(actor_scores_per_movie)

[1, 1, 0, 2, 0, 1, 2, 2, 0, 1, 1, 1, 0, 0, 2, 1, 0, 4, 3, 0, 0, 1, 0, 0, 3, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 3, 0, 0, 0, 0, 0, 3, 1, 2, 4, 2, 0, 0, 2, 2, 0, 1, 0, 2, 0, 2, 0, 1, 1, 2, 1, 1, 1, 2, 0, 2, 1, 1, 0, 0, 0, 0, 0, 2, 1, 0, 0, 0, 0, 3, 0, 3, 2, 0, 3, 1, 3, 0, 1, 0, 1, 0, 2, 2, 0, 0, 1, 0, 0, 2, 0, 2, 0, 2, 0, 1, 1, 3, 1, 1, 1, 0, 2, 0, 1, 0, 0, 2, 0, 0, 1, 2, 0, 0, 0, 1, 4, 1, 1, 1, 1, 0, 3, 3, 0, 4, 1, 2, 1, 2, 0, 3, 0, 2, 1, 2, 0, 1, 2, 1, 0, 1, 0, 1, 4, 2, 0, 4, 0, 3, 0, 0, 0, 0, 1, 0, 0, 4, 0, 4, 0, 1, 0, 2, 0, 3, 1, 0, 0, 1, 0, 2, 4, 0, 0, 0, 0, 0, 0, 0, 0, 1, 4, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 2, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 3, 1, 0, 1, 0, 4, 4, 1, 2, 0, 0, 0, 1, 0, 0, 1, 4, 0, 2, 0, 3, 1, 1, 1, 1, 0, 3, 0, 0, 3, 1, 0, 2, 0, 2, 4, 0, 4, 3, 3, 0, 0, 1, 3, 0, 1, 4, 3, 4, 0, 2, 0, 1, 4, 1, 4, 0, 0, 3, 1, 0, 0, 2, 1, 2, 0, 0, 0, 0, 1, 4, 0, 1, 0, 2, 1, 1, 2, 1, 3, 1, 0, 1, 2, 0, 1, 0, 0, 1, 0, 1, 2, 1, 3, 0, 1, 2, 2, 0, 2, 1, 3, 0, 0, 0, 2, 0, 3, 0, 0, 

In [22]:
movies_imdb=pd.DataFrame({'Title':movie_titles_imdb,'IMDB Rating':imdb_ratings,'MPAA Rating':MPAA_ratings,'Budget':movie_budgets,'Runtime':runtimes_imdb, 'Actor Score':actor_scores_per_movie, 'Action':Action,'Adventure':Adventure,'Animation':Animation,'Biography':Biography,'Comedy':Comedy,'Crime':Crime,'Drama':Drama,'Family':Family,'Fantasy':Fantasy,'History':History,'Horror':Horror,'Music':Music,'Mystery':Mystery,'Romance':Romance,'SciFi':SciFi,'Sport':Sport,'Thriller':Thriller,'War':War,'Western':Western})
movies_imdb.head()
movies_imdb.to_csv('movies_imdb_final.csv')

In [23]:
# Replace movie ratings with values
# If a TV rating e.g. if on netflix, replace with somewhat equivalent MPAA rating
movies_imdb['MPAA Rating']=movies_imdb['MPAA Rating'].replace('G',1).replace(['PG','TV-PG'], 2).replace(['PG-13','TV-14'],3).replace(['R','TV-MA'],4).replace([i for i in movies_imdb['MPAA Rating'] if i not in ['G','PG','PG-13','R']],np.nan)
movies_imdb.head()

,Title,IMDB Rating,MPAA Rating,Budget,Runtime,Actor Score,Action,Adventure,Animation,Biography,...,History,Horror,Music,Mystery,Romance,SciFi,Sport,Thriller,War,Western
0,Ted Lasso,8.8,4.0,,30.0,1,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
1,Free Guy,7.3,3.0,120000000,115.0,1,1,1,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Sex Education,8.3,4.0,,45.0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Only Murders in the Building,8.3,4.0,1851,NaN,2,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Seinfeld,8.8,2.0,22,22.0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [24]:
# Drop entries where no value for budget
movies_imdb = movies_imdb.replace({'Budget': {'':np.nan}})
print(movies_imdb)

                             Title  IMDB Rating  MPAA Rating     Budget  \
0                        Ted Lasso          8.8          4.0        NaN   
1                         Free Guy          7.3          3.0  120000000   
2                    Sex Education          8.3          4.0        NaN   
3     Only Murders in the Building          8.3          4.0       1851   
4                         Seinfeld          8.8          2.0         22   
...                            ...          ...          ...        ...   
9995                Director's Cut          6.1          NaN        NaN   
9996                          Ozzy          5.3          1.0        NaN   
9997                A Perfect Plan          6.3          NaN   16049338   
9998                      Landline          6.4          4.0        NaN   
9999                    Nature Cat          7.1          NaN         24   

      Runtime  Actor Score  Action  Adventure  Animation  Biography  ...  \
0        30.0          

In [26]:
movies_imdb.to_csv('movies_imdb.csv')